<a href="https://colab.research.google.com/github/mtcorreia/CI_AI_ADS/blob/main/Aula_11_e_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install lshw

In [ ]:
!cat /etc/os-release

# Usando IA para descoberta da ocorrência de Diabetes.

14 de Junho de 2024.

## Dataset do repositório de ML da UCI/Kaggle.


In [ ]:
# Importando os módulos.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
# Carregando o dataset.
df = pd.read_csv("pima-data.csv")

In [ ]:
# Verificando o formato dos dados.
df.shape

In [ ]:
df.describe

In [ ]:
# Verificando as primeiras linhas do dataset.
df.head()

In [ ]:
# Verificando as últimas linhas do dataset.
df.tail()

In [ ]:
# Verificando se existem valores nulos.
df.isnull().values.any()

In [ ]:
# Identificando a correlação entre as variáveis.
# OBS.: Correlação não implica em causalidade.
def plot_corr(df, size = 10):
  corr = df.corr()
  fig, ax = plt.subplots(figsize = (size, size))
  ax.matshow(corr)
  plt.xticks(range(len(corr.columns)), corr.columns)
  plt.yticks(range(len(corr.columns)), corr.columns)

In [ ]:
# Criando o gráfico.
plot_corr(df)

In [ ]:
# Visualização matricial da correlação.
# +1 = forte correlação positiva.
# 0 = não há correlação.
# -1 = forte correlação negativa.
df.corr()

In [ ]:
# Definindo as classes.
diabetes_map = {True: 1, False: 0}

In [ ]:
# Aplicando o mapeamento ao dataset.
df['diabetes'] = df['diabetes'].map(diabetes_map)

In [ ]:
# Verificando a alteração.
df.head(5)

In [ ]:
# Verificando como os dados estão distribuídos.
num_true = len(df.loc[df['diabetes'] == True])
num_false = len(df.loc[df['diabetes'] == False])
print("Número de Casos Verdadeiros: {0} ({1:2.2f}%)".format(num_true, (num_true/ (num_true + num_false)) * 100))
print("Número de Casos Falsos: {0} ({1:2.2f}%)".format(num_false, (num_false/ (num_true + num_false)) * 100))

## Spliting.
### 70% para dados de treino e 30% para dados de teste.

In [ ]:
from IPython.display import Image
Image('Treinamento.png')

In [ ]:
# Importando os módulos.
from sklearn.model_selection import train_test_split

In [ ]:
# Seleção de variáveis preditoras (Feature Selection)
atributos = ['num_preg', 'glucose_conc', 'diastolic_bp', 'thickness', 'insulin', 'bmi', 'diab_pred', 'age']

In [ ]:
# Variável a ser prevista.
atrib_prev = ['diabetes']

In [ ]:
# Criando objetos divididos pelos grupos.
X = df[atributos].values
Y = df[atrib_prev].values
X

In [ ]:
Y

In [ ]:
# Definindo a taxa de split.
split_test_size = 0.30

In [ ]:
# Criando dados de treino e de teste.
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size=split_test_size, random_state=44)

In [ ]:
X_treino.shape

In [ ]:
X_teste.shape

In [ ]:
Y_teste.shape

In [ ]:
Y_treino.shape

## Valores Missing Ocultos.

In [ ]:
# Verificando se existem valores nulos.
df.isnull().values.any()

In [ ]:
df.head()

In [ ]:
print("# Linhas no Dataframe {0}".format(len(df)))
print("# Linhas missing glucose_conc: {0}".format(len(df.loc[df['glucose_conc'] == 0])))
print("# Linhas missing diastolic_bp: {0}".format(len(df.loc[df['diastolic_bp'] == 0])))
print("# Linhas missing thickness: {0}".format(len(df.loc[df['thickness'] == 0])))
print("# Linhas missing insulin: {0}".format(len(df.loc[df['insulin'] == 0])))
print("# Linhas missing bmi: {0}".format(len(df.loc[df['bmi'] == 0])))
print("# Linhas missing age: {0}".format(len(df.loc[df['age'] == 0])))


## Tratando os dados missing ocultos.

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
# Criando o objeto.
preenche_0 = SimpleImputer(missing_values=0, strategy="mean")

# Substituindo os valores iguais a 0 pela média dos dados.
X_treino = preenche_0.fit_transform(X_treino)
X_teste = preenche_0.fit_transform(X_teste)

In [ ]:
X_treino

## 50% a 80% do tempo de trabalho de um Cientista de Dados é usado na exploração e preparação de dados.

In [ ]:
# Utilizando um classificador Naive Bayes.
from sklearn.naive_bayes import GaussianNB

In [ ]:
# Criando o modelo preditivo.
modelo_v1 = GaussianNB()

In [ ]:
# Treinando o modelo.
modelo_v1.fit(X_treino, Y_treino.ravel())

In [ ]:
# Verificando a exatidão no modelo de dados de treino.
from sklearn import metrics

In [ ]:
nb_predict_train = modelo_v1.predict(X_treino)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_treino, nb_predict_train)))
print()

In [ ]:
# Verificando a exatidão no modelo dos dados de Testes.
nb_predict_teste = modelo_v1.predict(X_teste)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, nb_predict_teste)))
print()

## Métricas e Matriz de Confusão.

In [ ]:
from IPython.display import Image
Image('ConfusionMatrix.jpg')

In [ ]:
# Criando uma Confusion Matrix.
print("Confusion Matrix")
print("{0}".format(metrics.confusion_matrix(Y_teste, nb_predict_teste), labels = [1, 0]))
print("")

print("Classification Report")
print(metrics.classification_report(Y_teste, nb_predict_teste))

## Otimizando o modelo com RandomForest.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
modelo_v2 = RandomForestClassifier(random_state=42)
modelo_v2.fit(X_treino, Y_treino.ravel())

In [ ]:
# Verificando a exatidão do Treino.
rf_predict_train = modelo_v2.predict(X_treino)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_treino, rf_predict_train)))
print()

In [ ]:
# Verificando a exatidão dos Testes.
rf_predict_teste = modelo_v2.predict(X_teste)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, rf_predict_teste)))
print()

In [ ]:
# Criando uma Confusion Matrix.
print("Confusion Matrix")
print("{0}".format(metrics.confusion_matrix(Y_teste, rf_predict_teste), labels = [1, 0]))
print("")

print("Classification Report")
print(metrics.classification_report(Y_teste, rf_predict_teste))

## Analisando o desempenho de mais um algoritmo de classificação: REGRESSÃO LOGÍSTICA.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Terceira versão do modelo usando Regrssão Logística.
modelo_v3 = LogisticRegression(C=0.7, random_state=42)
modelo_v3.fit(X_treino, Y_treino.ravel())

In [ ]:
# Verificando a exatidão dos Testes.
lr_predict_teste = modelo_v3.predict(X_teste)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, lr_predict_teste)))
print()

In [ ]:
# Criando uma Confusion Matrix.
print("Confusion Matrix:")
print("{0}".format(metrics.confusion_matrix(Y_teste, lr_predict_teste), labels = [1, 0]))
print("")

print("Classification Report:")
print(metrics.classification_report(Y_teste, lr_predict_teste))

## Summary
Resultado comparativo do desempenho dos algoritmo de classificação:

#### Modelo usando Algoritmo Naive Bayes (v1): 0.79
#### Modelo usando Algoritmo RandomForest (v2): 0.77
#### Modelo usando Algoritmo Regressão Logística (v3): 0.8


## Fazendo previsões com o modelo treinado (v3).

In [ ]:
import pickle

In [ ]:
# Salvando o modelo para usar na minha empresa.
filename = 'modelo_treinado_v3.sav'
pickle.dump(modelo_v3, open(filename, 'wb'))

In [ ]:
# Carregando o modelo e fazendo com novos conjunto de dados.
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(X_teste[15].reshape(1, -1))
result2 = loaded_model.predict(X_teste[18].reshape(1, -1))
print(result)
print(result2)

In [ ]:
X_treino[15]

In [ ]:
X_treino[18]

In [ ]:
df.head(20)